
# First-Order Logics & Planning

A number of classical planning representation languages such as `STRIPS` or `Functional STRIPS` have firm roots
in the logical tradition and use, to different degrees, the notion of
[first-order language](https://en.wikipedia.org/wiki/First-order_logic)
in order to succinctly encode the dynamics of the planning problem to be studied.
`Tarski` also builds on these ideas and makes a clear distinction between the first-order language
used to define a problem and the actual problem itself.


## Many-sorted languages

A many sorted logic is one in which the universe of discourse is divided into subsets, called *sorts*, rather than being an homogenous set. 
This is achieved by specifying $S$ a set of *sort symbols*, each of which denotes a non-empty set of the universe.

Definining sorts in `Tarski` is straightforward, we start instantiating the first-order language

In [ ]:
import tarski

In [ ]:
fol = tarski.language(theories=['equality', 'arithmetic'])

which will be acting as our facade to all things FOL.

In our model of _Blocks World_ we will consider two sorts, _block_ and _place_

In [ ]:
block = fol.sort('block')

In [ ]:
place = fol.sort('place')

`Tarski` allows to specify many sorted logics which only contemplate definitional hierarchies, hence sorts do not have _default_ symbols. Empty sorts are **not** allowed, and the `well_formed` method will raise an exception if a sort is found to be $\emptyset$.

In [ ]:
# uncomment the following line and execute this cell, you should get an exception of type LanguageError
#fol.check_well_formed()


### Providing sorts with content

Sorts are made of _constant symbols_, the following statement

In [ ]:
b1 = fol.constant('b1', block)

introduces the constant symbol _'b1'_ into sort _block_, which we have declared above. A language can have several sorts, with their own constants

In [ ]:
table = fol.constant('table', place)

We can declare a bunch of blocks easily too

In [ ]:
b2, b3 = [fol.constant( 'b_{}'.format(k), block ) for k in (2,3)]

by using a [generator expression](https://stackoverflow.com/questions/6416538/how-to-check-if-an-object-is-a-generator-object-in-python) to enumerate efficiently the names of the constants we want to introduce to sorts.

At any point, we can take a look at the contents of sorts declared

In [ ]:
fol.dump()['sorts']

as well as any _built-in_ sorts.

### Built-in sorts

Every language created with `Tarski` contains a number of _built-in_ sorts that allow modellers to account for algebraic relations and geometric concepts without having to define everything from first principles. At the time of writing this, every `Tarski` language comes with the following built-in sorts
 
  - `Real` - the set of real numbers $\mathbb{R}$
  - `Integer` - the set of integer numbers $\mathbb{Z}$
  - `Natural` - the set of natural numbers $\mathbb{N}$
  
These are represented as closed _intervals_ will well defined _lower_ and _upper_ bounds (the numbers specified in the domain). We cannot introduce symbols into built-in sorts, but we can _refer_ to them with Python variables

In [ ]:
x0 = fol.constant( 3, fol.Real )
print(x0)

In [ ]:
fol.Real.builtin

or

In [ ]:
magic = fol.constant( 42, fol.Integer)
print(magic)

or even

In [ ]:
import numpy as np
pi = fol.constant( np.pi, fol.Real)
print(pi)

### Intervals

So far the kind of sorts we have seen are either _sets_ defined arbitrarily, or an infinite subset of the ${\mathbb R}$ continuum (e.g. `Real` is actually a subset of the rationals ${\mathbb{Q}}$). One special case of sort is that which is a subset of the Cartesian product of its domain with itself, which we refer to as _intervals_. These subsets are assumed to be _dense_, and allow to represent without loss of generality sorts like

$$
I \equiv [a,b],\ \text{where}\, a,b \in {\mathbb Z}
$$

or some other built-in sort.

We can declare the interval sort for $[0,10]$ where $0$ and $10$ are taken to be integers as follows

In [ ]:
I = fol.interval('I', fol.Integer, 0, 10)

Constants are then given as in the unrestricted case

In [ ]:
one = fol.constant(1, I)

Trying to creat constants of interval types which aren't consistent with the declared bounds will result in an exception

In [ ]:
try:
    fol.constant(-2, I)
except ValueError as err:
    print("Caught exception: {}".format(err))

### A Hierarchy of Sorts

Sorts associated to a language can be arranged as per hierarchy, specifying the partial ordering relation $\sqsubseteq$ to hold between two given sorts $\alpha$ and $\beta$

In [ ]:
alpha = fol.sort('alpha')

In [ ]:
beta = fol.sort('beta', alpha)

The _parent_ of $\beta$, that is, the sort $\alpha$ s.t. $\alpha \sqsubseteq \beta$, is accessible via the method `parent`

In [ ]:
print("Parent of {} is {}".format(beta, tarski.syntax.sorts.parent(beta)))

For built-in sorts, this relationship is already defined as expected

In [ ]:
R = fol.Real

print("Parent of {} is {}".format(R, tarski.syntax.sorts.parent(R)))

In [ ]:
Z = fol.Integer

print("Parent of {} is {}".format(Z, tarski.syntax.sorts.parent(Z)))

In [ ]:
N = fol.Natural

print("Parent of {} is {}".format(N, tarski.syntax.sorts.parent(N)))

## Constants, Functions and Predicates

Let's start populating the language `bw` for describing instances of Blocks World

In [ ]:
import tarski
import tarski.errors as err

In [ ]:
bw = tarski.language()

Blocks Worlds are made of objects of two sorts:

In [ ]:
place = bw.sort('place')

and

In [ ]:
block = bw.sort('block', place)

We populate our instance with a few blocks

In [ ]:
b1, b2, b3, b4 = [ bw.constant('b_{}'.format(k), block )  for k in (1,2,3,4) ]

and a table

In [ ]:
table = bw.constant('table', place)

### Functions

Function symbols $f$ are used to represent mappings between several sorts $\tau$. Formally, we will define $f$ as mappings

$$
f : \tau_1, \ldots, \tau_n \mapsto \tau_{n+1}
$$

Functions $f$ have _arity_ $n \geq 0$, their _domain_ is the cartesian product $\tau_1 \times \tau_2 \ldots \times \tau_n$ and their _codomain_ is the sort $\tau_{n+1}$. The _signature_ $\sigma_f$ of $f$ corresponds with the tuple

$$
(f, \tau_1, \ldots, \tau_n, \tau_{n+1})
$$

and allows to uniquely identify a function: `Tarski` doesn't allow languages with functions $f$ and $f'$ such that $\sigma_f$ $=$ $\sigma_{f'}$.

For Blocks World we can define the function $loc: block \mapsto place$, which we use to refer indirectly to the object a given block is _on top of_ at any point in time

In [ ]:
loc = bw.function('loc', block, place)

We note that the arguments of this method correspond with the components of a function signature, hence

In [ ]:
print('Domain of {}: {}'.format(loc, loc.domain))
print('Codomain of {}: {}'.format(loc, loc.codomain))
print('Type of {}: {}'.format(loc, loc.sort))
print('Arity of {} : {}'.format(loc, loc.arity))

Printing function objects indicates the arity (number of arguments) the function was declared with, following the convention typically used in Prolog.


### Predicates as Relation Symbols
Relations between objects and intrinsic properties of objects are modelled by means of _relation symbols_ or _predicates_.

In [ ]:
clear = bw.predicate('clear', block )

By default, `Tarski` languages do not define implictly any kind of builtin predicate or function. For instance, 
if we try to write something like

In [ ]:
try:
    b1 == b2
except err.LanguageError as e:
    print(f"Caught exception {e}")
    

For that we need to explicitly attach _theories_ to our language, as shown later.


## Terms and Formulas

Now we have all the elements to formally define `Tarski` languages:

**Definition** (Many-Sorted First-Order Language). A _many-sorted_ _first-order_ language ${\cal L}$ is made up of:
 - A non-empty set $T$ of _sorts_
 - An _infinite number_ of _variables_ $x_{1}^{\tau}, x_{2}^{\tau}, \ldots$ for each short $\tau \in T$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n+1}$ of sorts, a (possibly empty) set of _function_ symbols, each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n+1})$
 - For each $n \geq 0$ and each tuple $(\tau_1, \ldots, \tau_{n+1}) \in T^{n}$ of sorts, a (possibly empty) set of _relation_ symbols (predicates), each of which is said to have _arity_ and _type_ $(\tau_1, \ldots, \tau_{n})$

Continuing with our `Blocks World` themed example

In [ ]:
import tarski
from tarski.syntax import *
from tarski.theories import Theory

# 1. Create language used to describe world states and transitions
bw = tarski.language(theories=[Theory.EQUALITY, Theory.ARITHMETIC])

# 2. Define sorts
place = bw.sort('place')
block = bw.sort('block', place)

# 3. Define functions
loc = bw.function( 'loc', block, place )
looking_at = bw.function( 'looking_at', block )

# 4. Define predicates
clear = bw.predicate( 'clear', block)

We introduce the function $width(b)$ for blocks $b$, this will allow us to specify Hanoi Towers like tasks

In [ ]:
width = bw.function('width', block, bw.Real)

_Constants_ are 0-arity functions, whose sort $\tau$ is a set with one single element. Hence, we handle them separately, as we specialise their representation

In [ ]:
# 5. Define constants
b1, b2, b3, b4 = [ bw.constant('b_{}'.format(k), block) for k in (1,2,3,4) ]
table = bw.constant('table', place)

### (First-Order) Terms

Combinations of variables, functions and constants are called _terms_, and the rules for constructing them are given inductively:

**Definition** (First-Order Terms). A term $t$ can be:

 - Any variable $x^{\tau}$ of the language can be a term $t$ with type $\tau$
 - Any constant symbol of the language with type $\tau$ is a term with the same type
 - If $t_1, \ldots, t_n$ are terms with respective types $\tau_1, \ldots, \tau_n$ and $f$ is a _function_ symbol with type $(\tau_1, \ldots, \tau_n, \tau{n+1})$ then $f(t_1,\ldots,t_n)$ is a term with type $\tau_{n+1}$.

Terms are implemented as Python objects. Every constant symbol is an instance of `Term`

In [ ]:
from tarski import Term

isinstance(b1,Term)

Function symbols allow to nest terms, thus 

In [ ]:
t1 = loc(b1)
isinstance(t1,Term)

In [ ]:
x = bw.variable('x', block)
t2 = loc(x)
isinstance(t2,Term)

In [ ]:
t3 = loc(looking_at())
isinstance(t3,Term)

are all terms. `Tarski` textual representation of variables is a bit different

In [ ]:
print('{}, type: {}'.format(t1, t1.sort))
print('{}, type: {}'.format(t2, t2.sort))
print('{}, type: {}'.format(t3, t3.sort))

in order to make distinct variables from constants, the former are printed with the prefix `?`. 

### Formulas

Formulas (statements that can be either `True` or `False`) are defined also inductively as follows:

**Definition** (First-Order Formulas).

 - If $t_1$ and $t_2$ are two terms with the same type, then $t_1 = t_2$ is an _atomic formula_.
 
 - If $t_1,\ldots,t_n$ are terms with respective types $\tau_1,\ldots,\tau_n$, and $R$ is a relation symbol with type $(\tau_1,\ldots,\tau_n)$, then $R(t_1,\ldots,t_n)$ is an atomic formula too.
 
 - If $\phi_1$ and $\phi_2$ are formulas then $\neg \phi_1$, $\phi_1 \lor \phi_2$ and $\phi_1 \land \phi_2$ are also formulas.
 
 - If $\phi$ is a formula, then $\exists_t x^{\tau}\, \phi$ and $\forall_t x^{\tau}\, \phi$ are also formulas.

Quantification happens over a certain sort, i.e. for each sort $\tau$ $\in$ $T$ there are universal and existential quantifier symbols $\forall_{\tau}$ and $\exists_{\tau}$, which may be applied to variables of the same sort.

Formulas without existential ($\exists$) or universal ($\forall$) quantifiers are called _quantifier free_.

### Examples

We can define the formula $t_1 = t_3$ - terms $t_1$ and $t_3$ are equal - with the following statement

In [ ]:
tau = t1 == t3

The `str()` method is overwritten for every term and formula class, returning a string representation of expression, which gives insight into how Tarski represents internally formulas and expressions

In [ ]:
str(tau)

We need a new variable so we can make general statements about more than one block

In [ ]:
y = bw.variable('y', block)

Now we can state properties of states like _for every block x, x cannot be wider than the place below_

$$
\forall x,y\, loc(x) = y \supset width(x) < width(y)
$$

which can be written as

In [ ]:
phi = forall( x, y, implies( loc(x) == y, width(x) < width(y) ) )

which is represented internally

In [ ]:
str(phi)

It's worth noting that Tarski will always try to simplify formulas. For instance, the sub-formula 

$$ 
loc(x) = y \supset width(x) < width(y)
$$

was transformed into the disjunction

$$
loc(x) \neq y \lor width(x) < width(y)
$$

using the transformation 

$$
p \supset q \equiv \neg p \lor q
$$

We can use the operator `>` instead of the function `implies()`, if a more concise syntax is preferred.

In [ ]:
phi = forall( x, y, (loc(x) == y) > (width(x) < width(y)) )

We can write the conjunctive  formula

$$
loc(b1) \neq loc(b2) \land loc(b1) \neq loc(b3)
$$

in several ways. One is using the `land` function

In [ ]:
phi = land( loc(b1) != loc(b2), loc(b1) != loc(b3))

or the operator `&`

In [ ]:
phi = (loc(b1) != loc(b2)) & (loc(b1) != loc(b3))

Another state invariant like 

$$
loc(b1) = b2 \lor loc(b1) = b3
$$

can be written as

In [ ]:
phi = lor( loc(b1) == b2, loc(b1) == b3 )

or

In [ ]:
phi = (loc(b1)==b2) | (loc(b1)==b3)

Finally, the formula 

$$
loc(b1) = b2 \supset \neg clear(b2)
$$

can be written as 

In [ ]:
phi=implies( loc(b1) == b2, neg(clear(b2)))
str(phi)

or, alternatively the `~` unary operator can be used instead of `neg`

In [ ]:
phi = implies( loc(b1) == b2, ~clear(b2))
str(phi)

